In [1]:
# hyperloglog

import numpy as np
import spookyhash
import mmh3
import os
import pandas as pd
import time
import operator
import hyperloglog
import sys

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=get_emax()
    col,row=position(element)
        # col / row index of element 
    avg=0
    #print("{} send to Sk[{}][{}]".format(element,row,col))
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    Sk_head[row].distinct.add(element.ID)
    Sk[row][col]+=1
    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk,row)

'''
    print("e_max:{}".format(e_max))
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print('')
'''

# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # local max need only 1 row
    #print("In Update_local_max:")
    width,depth=get_width_depth()
    if head_item.maxID=='':
        head_item.maxID=element.ID
    else:
        # local_max_col=(mmh3.hash(head_item.maxID,signed=False))% ((width*numerator)//denominator)
        local_max_col=(mmh3.hash(head_item.maxID,signed=False))% width
        if element_list[local_max_col]<element_list[column]:
            head_item.maxID=element.ID        
            

# ==========================update e_max==========================
def Update_emax(head,sketch,sk_row):
    '''
    # pass whole array
    #print("In Update_emax:")
    e_max=get_emax()
    width,depth=get_width_depth()
    for i in range(len(head)):
        if head[i].maxID=='':
            continue
        else:
            local_max_col,local_max_row=position(Tail(head[i].maxID,0))
            if sketch[local_max_row][local_max_col]>e_max.count:
                e_max.ID=head[i].maxID
                e_max.count=sketch[local_max_row][local_max_col]        
    '''
    e_max=get_emax()
    local_max_col,local_max_row=position(Tail(head[sk_row].maxID,0))
    if sketch[local_max_row][local_max_col]>e_max.count:
        e_max.ID=head[sk_row].maxID
        e_max.count=sketch[local_max_row][local_max_col]
# ========================== BringBack=========================
def BringBack(e_min,head,sketch):
    # bring e_max back to Top
    # e_min=e_max, e_max=Null, delete e_max.count in Sketch, send e_min into Sketch
    e_max=get_emax()
    temp=Tail(e_min.ID,e_min.count)
    e_min.ID=e_max.ID
    e_min.count=e_max.count
    DeleteSk(e_max,head,sketch)
    UpdateSk(temp,head,sketch)

# ==========================DeleteSk=========================
def DeleteSk(element,head,sketch):
    # e_max in sketch: sketch[r][c]=0, total count-=sketch[row][col]
    #width,depth=get_width_depth()
    col,row=position(element)
    head[row].count-=e_max.count
        # total_count-=element.count
    sketch[row][col]=0
    head[row].maxID=''
    element.ID=""
    element.count=0
# ==========================Tools=========================    
def get_emax():
    return e_max
def get_width_depth():
    return width,depth

def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    #numerator,denominator=get_fraction()
    width,depth=get_width_depth()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input: byte
        # output:unsigned- 32 bit int
    hash2=mmh3.hash(element.ID, signed=False)
    #hash2=mmh3.hash(str(hash1), signed=False)
        # input: str
        # output: unsigned- 32 bit int
    #col=hash2 % ((width*numerator)//denominator)
    col=hash2 % width
    row=hash1 % depth
    return col,row
def get_fraction():
    return numerator,denominator    
    
# ==========================main=========================    

filename='caida_0.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=256
width=256
size=4096
numerator=1
denominator=1

start=time.time()

Sk_head=[Head(0) for j in range(depth)]
Sketch=np.zeros((depth,width),dtype='int32')
e_max=Tail('',0)
Top=[]

item_count=1000
income=0
with open(src_data,'rb') as file:
     #以binary讀取，資料型態也為byte
    while True:
        e=str(file.read(13))
        if len(e)<13:
            print('EOF')
            break
        else:
            #item_count-=1
            #income+=1
            #print("read {}-th element:{}".format(income,e))
            item=Tail(e,1)            
            index=find(item,Top)
            if index<0:
                if len(Top)<size:
                    Top.append(item)
                else:
                    UpdateSk(item,Sk_head,Sketch)
            else:
                Top[index].count+=1
                if index==0 or Top[index].count< Top[index-1].count:
                    pass
                else:
                    Top.sort(key=operator.attrgetter('count'),reverse=True)                      
        if e_max.count>Top[-1].count:
            BringBack(Top[-1],Sk_head,Sketch)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            #print('Top after BringBack: \n\t{}'.format(Top)) 

end=time.time()
print("Top-{},Sketch:{}*{}".format(size,depth,width))
print("Execution time:{:8.3f} seconds.".format(end-start))
print("Total memory {} bytes=".format(sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth),end='')
print("Top:{} bytes, Sketch:{} bytes, Sketch_head:{} bytes.".format(sys.getsizeof(Top),Sketch.nbytes,sys.getsizeof(Sk_head[0])*depth))

'''
print("TOP[20]:\n{}".format(Top[:20]))
print("e_max:{}".format(e_max))
for i in range(len(Sketch)):
    print("Sk[{}]:{},{}".format(i,Sk_head[i],Sketch[i]))
print('')

'''
#====================Top to csv=============================
templi=[[i.ID,i.count] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])

path="..\\Caida0\\"
name="My_caida0_distinct"+'_'+str(size)+'_'+str(depth)+'_'+str(width)
final=name+".csv"
df.to_csv(path+final,index=False)
# ====================precision====================
groundtruth='caida_0_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(path,groundtruth))
    # compare with Top-k and groundtruth[k]
My_result=pd.read_csv(os.path.join(path,final))

# precision
gt_set=set(grtruth['Element'][:size])
my_set=set(My_result['ID'])
precision=len(gt_set & my_set)/len(my_set)
    # &: set 交集運算
    # precision= tp/tp+fp=tp/size
print("Precision: {}".format(precision))

# ====================ARE/AAE of Top====================
gt_li=grtruth.values.tolist()[:size]
top_li=My_result.values.tolist()
ID=[j[0] for j in gt_li]
top_are=0
top_aae=0
tp=0
fp=0
for item in top_li:
    if item[0] in ID:
        tp+=1
        index=ID.index(item[0])
        top_are+=abs(gt_li[index][1]-item[1])/gt_li[index][1]
        top_aae+=abs(gt_li[index][1]-item[1])
    else:
        fp+=1
top_are=top_are/size
top_aae=top_aae/size
# print(top_are,top_aae)    
    
# ====================ARE/AAE of all====================
gt_dict=dict(grtruth.values.tolist())
top_dict=dict(My_result.values.tolist())
distinct=len(gt_dict)
    # cardinality of all incoming elements
row_cardinality=[len(i.distinct) for i in Sk_head]

all_are_error=0
all_aae_error=0
for item in gt_dict:
    if item in top_dict:
        # item in Top
        all_are_error+=abs(top_dict[item]-gt_dict[item])/gt_dict[item]
        all_aae_error+=abs(top_dict[item]-gt_dict[item])
    else:
        # item in Sketch
        item_col,item_row=position(Tail(item,1))
        ratio=width/row_cardinality[item_row]
        estimate=0
        if Sketch[item_row][item_col]*ratio<1:
            estimate=1
        all_are_error+=abs(estimate-gt_dict[item])/gt_dict[item]
            # 此dataset暫無count為0的情況
        all_aae_error+=abs(estimate-gt_dict[item])

all_ARE=all_are_error/distinct
all_AAE=all_aae_error/distinct
print("Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_ARE))
print("all_AAE: {:8.6f}".format(all_AAE))

# ====================Cover result into a dataframe====================
sketch_size=str(depth)+'*'+str(width)
temp=sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth
memory_usage=str(temp)+' bytes ='+'Top:'+str(sys.getsizeof(Top))+'+ Sketch:'+str(Sketch.nbytes)+'+ Sk_head:'+str(sys.getsizeof(Sk_head[0])*depth)

result_df=pd.DataFrame(columns=['Top-k',
                                'Sketch',
                                'Total memory',
                                'Exe_time',
                                'Find',
                                'Precision',
                                'ARE-Top',
                                'AAE-Top',
                                'ARE-all',
                                'AAE-all'])
output_dict={
    'Top-k':size,
    'Sketch':sketch_size,
    'Total memory':memory_usage,
    'Exe_time':float('{:.3f}'.format(end-start)),
    'Find':"Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp),
    'Precision':float("{:8.4f}".format(precision)),
    'ARE-Top':float('{:8.6f}'.format(top_are)),
    'AAE-Top':float('{:8.6f}'.format(top_aae)),
    'ARE-all':float('{:8.6f}'.format(all_ARE)),
    'AAE-all':float('{:8.6f}'.format(all_AAE))}
result_df=result_df.append(output_dict,ignore_index=True)
file="My_caida0_distinct"+'_'+str(size)+'_'+str(depth)+'_'+str(width)+'_.csv'
result_df.to_csv(file,index=False)
result_df

EOF
Top-4096,Sketch:256*256
Execution time:1789.836 seconds.
Total memory 310408 bytes=Top:33928 bytes, Sketch:262144 bytes, Sketch_head:14336 bytes.
Precision: 0.8984375
Find:3680, TP:3680, FP:416
Top_ARE: 0.026032
Top_AAE: 3.745850
all_ARE: 2.054308
all_AAE: 5.518346


,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,4096,256*256,310408 bytes =Top:33928+ Sketch:262144+ Sk_hea...,1789.836,"Find:3680, TP:3680, FP:416",0.8984,0.026032,3.74585,2.054308,5.518346


[16889, 11647, 10113, 8993, 8418, 6596, 6244, 5420, 5194, 5044, 4444, 4246, 4081, 4081, 4054, 3972, 3767, 3638, 3588, 3584, 3425, 3419, 3407, 3292, 3284, 3231, 3140, 3126, 3123, 3033, 3000, 2988, 2985, 2983, 2983, 2946, 2892, 2891, 2865, 2859, 2761, 2740, 2724, 2701, 2698, 2694, 2644, 2617, 2569, 2546, 2545, 2540, 2537, 2531, 2530, 2520, 2508, 2501, 2498, 2442, 2436, 2416, 2382, 2372, 2369, 2341, 2321, 2307, 2280, 2257, 2229, 2229, 2223, 2209, 2208, 2157, 2136, 2113, 2102, 2087, 2086, 2079, 2077, 2076, 2073, 2055, 2047, 2040, 2034, 2023, 2001, 1992, 1956, 1946, 1945, 1941, 1891, 1887, 1883, 1880, 1876, 1871, 1854, 1839, 1836, 1835, 1818, 1814, 1797, 1796, 1787, 1777, 1767, 1766, 1752, 1734, 1733, 1729, 1716, 1716, 1704, 1702, 1701, 1701, 1699, 1686, 1672, 1668, 1650, 1644, 1631, 1628, 1628, 1613, 1610, 1604, 1574, 1572, 1564, 1550, 1539, 1535, 1515, 1511, 1504, 1501, 1485, 1484, 1482, 1482, 1481, 1465, 1462, 1458, 1451, 1447, 1446, 1440, 1421, 1408, 1408, 1400, 1394, 1390, 1376, 1373, 

In [75]:
path="..\\Caida0\\"
groundtruth='caida_0_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(path,groundtruth))

y=grtruth.groupby(by='Count')
z=y.size()
    # only 139 different count for this dataset
print(z.index.tolist())
    # e出現的頻率
print(z.values.tolist())
    # 不同頻率出現的次數

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [81]:
#print(z.index.tolist())
    # e出現的頻率
#print(z.values.tolist())
    # 不同頻率出現的次數

xdf=pd.DataFrame(columns=['Frequency','Number'])
xdf['Frequency']=z.index.tolist()
xdf['Number']=z.values.tolist()
xdf

,Frequency,Number
0,1,57963
1,2,25835
2,3,17985
3,4,12341
4,5,9441
...,...,...
957,8418,1
958,8993,1
959,10113,1
960,11647,1
